# Unsupervised Learning / Data Analytics 1
The purpose of this notebook is to .....analyse user behaviour and network characteristics.
outliers in every section individual

## Structure
1. User Analysis
    1. Number of unique users
    2. Most active users
    3. Distribution of user activity
    4. Temporal patterns in user activity (e.g., daily/weekly trends)
    5. Identification of outlier users
2. Network Analysis
    1. Network density and sparsity
    2. Community detection using Louvain algorithm
    3. Centrality measures (e.g., degree, betweenness, closeness)
    4. Identification of key users (bridges, hubs, and influencers)
    5. Clustering coefficients and neighborhood connectivity

## Summarized Results
- The network is globally sparse, with low average clustering, meaning most users are not well-connected to their neighbors.
- Within this sparse structure, some communities form tightly-knit clusters, which the Louvain algorithm identifies.
- Central users, such as `1010626427`, act as bridges connecting these otherwise independent communities, creating a sparse but connected global network.
- User activity distribution is highly skewed, with a few users contributing disproportionately to the total activity.

# Overview

Load the data

# Network Analysis

# Post Analysis

### Temporal Analysis

# User Analysis